## Load data

In [1]:
!wget  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
!unzip data_object_label_2.zip -d ./data_object_label_2 | grep "I love DL"
!rm data_object_label_2.zip 

--2020-12-09 11:35:41--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.75.79
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.75.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5601213 (5.3M) [application/zip]
Saving to: ‘data_object_label_2.zip’

data_object_label_2 100%[===================>]   5.34M  6.65MB/s    in 0.8s    

2020-12-09 11:35:43 (6.65 MB/s) - ‘data_object_label_2.zip’ saved [5601213/5601213]



In [2]:
import os
import cv2
import math
import random
import torch.utils.data as data
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import os

from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.model_selection import train_test_split
import gc; gc.enable() # memory is tight
import torch

dtype = "float32"

## Create file path list

In [4]:
#load list 
!wget https://xiaozhichen.github.io/files/mv3d/imagesets.tar.gz

--2020-12-09 11:35:58--  https://xiaozhichen.github.io/files/mv3d/imagesets.tar.gz
Resolving xiaozhichen.github.io (xiaozhichen.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to xiaozhichen.github.io (xiaozhichen.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163840 (160K) [application/gzip]
Saving to: ‘imagesets.tar.gz’

imagesets.tar.gz    100%[===================>] 160.00K  --.-KB/s    in 0.03s   

2020-12-09 11:35:58 (5.26 MB/s) - ‘imagesets.tar.gz’ saved [163840/163840]



In [5]:
#load eval code
!git clone https://github.com/cguindel/eval_kitti.git


Cloning into 'eval_kitti'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 83 (delta 8), reused 11 (delta 4), pack-reused 63
Unpacking objects: 100% (83/83), done.


In [6]:
!cd eval_kitti && mkdir build && cd build && cmake .. && make

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Found PNG: /usr/lib/x86_64-linux-gnu/libpng.so (found version "1.6.34") 
-- Configuring done
-- Generating done
-- Build files have been written to: /content/eval_kitti/build
Scanning dependencies of target evaluate_object
[ 50%] Building CXX object CMakeFiles/evaluate_object.dir/evaluate_object.cpp.o
[100%] Linking CXX execut

In [7]:
!cd eval_kitti/build && mkdir data && cd data && mkdir object

In [8]:
!cp -r ./data_object_label_2/training/label_2/ eval_kitti/build/data/object

In [11]:

!tar -xvf imagesets.tar.gz 
!mv ./ImageSets ./lists
!mv ./lists ./eval_kitti/build/lists

ImageSets/
ImageSets/test.txt
ImageSets/train.txt
ImageSets/val.txt
ImageSets/trainval.txt


In [14]:
# !rm -rf eval_kitti/build/results/exp
!mkdir eval_kitti/build/results && cd eval_kitti/build/results  && mkdir exp && cd exp 


In [16]:
!cp -r ./eval_kitti/build/data/object/label_2/ eval_kitti/build/results/exp/
!mv eval_kitti/build/results/exp/label_2 eval_kitti/build/results/exp/data

In [17]:
for i in range(7481):

  filepath = "/content/eval_kitti/build/results/exp/data/" + str(i).zfill(6)  + ".txt"

  with open(filepath) as fp:
      lines = fp.read().splitlines()
  with open(filepath, "w") as fp:
      for line in lines:
        if line[:3] == "Car":
          print(line + " 1.00", file=fp)

In [18]:
!cd eval_kitti/build && ./evaluate_object exp trainval

Starting evaluation...
Results list: lists/trainval.txt
Getting valid images... 
File loaded
Loading detections... 
-----------
GT STATS
-----------
car : 28742
pedestrian : 4486
cyclist : 1627
van : 2914
truck : 1094
person_sitting : 222
tram : 511
-----------
DET STATS
-----------
car : 28742
pedestrian : 0
cyclist : 0
van : 0
truck : 0
person_sitting : 0
tram : 0
  done.
car: Starting 2D evaluation (Easy/Moderate/Hard)...
Getting detection scores to compute thresholds
Evaluating on 5968 valid ground-truth instances and 17082 valid detections
Computing statistics
Stats computed 
Getting detection scores to compute thresholds
Evaluating on 15706 valid ground-truth instances and 24634 valid detections
Computing statistics
Stats computed 
Getting detection scores to compute thresholds
Evaluating on 21710 valid ground-truth instances and 24634 valid detections
Computing statistics
Stats computed 
Saving results/exp/plot/car_detection.txt
-----------
car_detection AP (%): 100.00 / 100.00 

In [20]:
!python ./eval_kitti/parser.py exp

--car detection--
Easy	Mod.	Hard
1.0000	1.0000	1.0000
-----------------------



In [ ]:
!rm -rf eval_kitti/build/

